<a href="https://colab.research.google.com/github/aravind598/Data_Science_Proj/blob/master/Data%20Science%20Project/Deep_Learning_EarthQuake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://github.com/Goodsea/Richter-s-Eye/blob/master/main.ipynb

In [13]:
!pip install tensorflow==1.1
import numpy as np
import pandas as pd

import lightgbm as lgb

import keras 
from keras.layers import *
from keras.optimizers import *
from keras.models import Model

from sklearn.metrics import f1_score 
from sklearn.model_selection import KFold
!pip install tensorflow==1.1

In [0]:
url = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_labels.csv?token=ANW7SFBKZIJTONPGC67MZMS6SCJR2'
urll = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/train_values.csv?token=ANW7SFBK2YMNTYBWW44PE7C6SCJSE'
url2 = 'https://raw.githubusercontent.com/aravind598/Github/aravind598/test_values.csv?token=ANW7SFHJZIY2XRWW5QVSW4S6SCS5I'

In [0]:
train_x = pd.read_csv(urll)
train_y = pd.read_csv(url)
test_x  = pd.read_csv(url2)

In [0]:
geo1 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_1_id"], test_x["geo_level_1_id"]])))
geo2 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_2_id"], test_x["geo_level_2_id"]])))
geo3 = np.array(pd.get_dummies(pd.concat([train_x["geo_level_3_id"], test_x["geo_level_3_id"]])))

In [7]:
geo3

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [60]:
def NET():
    inp = Input((geo3.shape[1],))
    i1 = Dense(22, name="intermediate")(inp)
    x2 = Dense(geo2.shape[1], activation='sigmoid')(i1)
    x1 = Dense(geo1.shape[1], activation='sigmoid')(i1)

    model = Model(inp, [x2,x1])
    model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
    return model

model = NET()
model.fit(geo3, [geo2, geo1], batch_size=128, epochs=15, verbose=1)

Epoch 1/21
347469/347469 [==============================] - 125s 358us/step - loss: 0.1425 - dense_17_loss: 0.0347 - dense_18_loss: 0.1078 - dense_17_acc: 0.9987 - dense_18_acc: 0.9742
Epoch 2/21
347469/347469 [==============================] - 125s 359us/step - loss: 0.0142 - dense_17_loss: 0.0044 - dense_18_loss: 0.0098 - dense_17_acc: 0.9993 - dense_18_acc: 0.9979
Epoch 3/21
347469/347469 [==============================] - 127s 365us/step - loss: 0.0051 - dense_17_loss: 0.0035 - dense_18_loss: 0.0016 - dense_17_acc: 0.9993 - dense_18_acc: 0.9997
Epoch 4/21
347469/347469 [==============================] - 126s 363us/step - loss: 0.0032 - dense_17_loss: 0.0027 - dense_18_loss: 4.8027e-04 - dense_17_acc: 0.9994 - dense_18_acc: 0.9999
Epoch 5/21
347469/347469 [==============================] - 126s 363us/step - loss: 0.0016 - dense_17_loss: 0.0014 - dense_18_loss: 1.6002e-04 - dense_17_acc: 0.9996 - dense_18_acc: 1.0000
Epoch 6/21
347469/347469 [==============================] - 124s 35

In [0]:
# "Extract Intermediate Layer" Function
from keras import backend as K

get_int_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [0]:
# Extract GEO-Embeds for all train data points.
# Then assign with train_data

out = []
for dat in geo3[:260601]:
    layer_output = get_int_layer_output([[dat]])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

train_data = pd.get_dummies(train_x.copy())
train_data = train_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
train_data = train_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15],
                               geo_feat17=out[:,16],
                               geo_feat18=out[:,17],
                               geo_feat19=out[:,18],
                               geo_feat20=out[:,19],
                               geo_feat21=out[:,20],
                               geo_feat22=out[:,21])'''
                               geo_feat23=out[:,22],
                               geo_feat24=out[:,23],
                               geo_feat25=out[:,24],
                               geo_feat26=out[:,25],
                               geo_feat27=out[:,26],
                               geo_feat28=out[:,27],
                               geo_feat29=out[:,28],
                               geo_feat30=out[:,29],
                               geo_feat31=out[:,30],
                               geo_feat32=out[:,31],
                               geo_feat33=out[:,32],
                               geo_feat34=out[:,33],
                               geo_feat35=out[:,34],
                               geo_feat36=out[:,35],
                               geo_feat37=out[:,36],
                               geo_feat38=out[:,37],
                               geo_feat39=out[:,38],
                               geo_feat40=out[:,39],
                               geo_feat41=out[:,40],
                               geo_feat42=out[:,41],
                               geo_feat43=out[:,42],
                               geo_feat44=out[:,43],
                               geo_feat45=out[:,44],
                               geo_feat46=out[:,45],
                               geo_feat47=out[:,46],
                               geo_feat48=out[:,47],
                               geo_feat49=out[:,48],
                               geo_feat50=out[:,49],
                               geo_feat51=out[:,50],
                               geo_feat52=out[:,51])    '''               

In [63]:
train_data.head()

,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_n,roof_type_q,roof_type_x,ground_floor_type_f,...,geo_feat13,geo_feat14,geo_feat15,geo_feat16,geo_feat17,geo_feat18,geo_feat19,geo_feat20,geo_feat21,geo_feat22,geo_feat23,geo_feat24,geo_feat25,geo_feat26,geo_feat27,geo_feat28,geo_feat29,geo_feat30,geo_feat31,geo_feat32,geo_feat33,geo_feat34,geo_feat35,geo_feat36,geo_feat37,geo_feat38,geo_feat39,geo_feat40,geo_feat41,geo_feat42,geo_feat43,geo_feat44,geo_feat45,geo_feat46,geo_feat47,geo_feat48,geo_feat49,geo_feat50,geo_feat51,geo_feat52
0,802906,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1.355477,-1.215669,0.200620,-0.583732,-0.875989,0.079356,-0.799342,0.383651,-0.969284,1.602275,-0.567598,-0.680350,0.705669,-0.052653,0.844732,-0.590197,-0.370399,-0.904696,-0.359983,-1.415984,1.222670,0.794824,1.257459,-0.943929,1.196352,0.800284,1.103202,0.913038,-1.077533,0.826017,-0.324469,-0.047996,0.891431,-0.319790,0.565639,-0.005926,1.281738,0.480955,-1.652525,0.037840
1,28830,2,10,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,1.379600,-1.002091,-0.645644,-0.824387,-0.227066,0.584196,0.518741,-0.967274,-1.078113,-0.133461,-0.860669,0.062079,0.994265,-0.559878,0.240659,0.213234,0.457159,-0.795387,0.699039,-1.526299,1.090839,-0.352884,0.871344,-0.676990,0.939544,1.023179,-0.010579,-0.346210,-1.336603,1.051179,-0.834154,0.852214,-0.147973,-0.337790,0.429382,1.321793,0.845404,-0.611602,-1.132260,0.605828
2,94947,2,10,5,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,0.006292,-1.150647,-0.885030,-0.647391,-0.676553,-0.388415,-1.085834,-1.263406,0.115893,1.414679,-0.784728,-0.790691,-0.598627,-0.952435,-0.470089,-0.410256,-0.534014,-1.465700,-0.633737,-1.115990,0.948649,0.728365,1.016477,-0.472688,1.109047,0.815777,1.467325,1.095979,-1.114978,0.706553,-1.480441,1.235057,0.730436,0.629157,0.545954,1.171672,1.414156,0.653624,-1.631732,1.159220
3,590882,2,10,6,5,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1.539507,-0.276203,-0.572600,0.511721,-0.253085,-0.702741,-1.057034,-0.769530,-0.973195,0.306001,-0.827571,-1.121453,0.734296,-1.198127,-0.308170,0.705398,0.714098,-1.081913,0.069117,-0.842954,1.104589,0.716877,1.118392,0.150132,-0.349556,1.015395,1.218751,-0.536609,-1.002840,0.695570,-1.100745,1.277677,0.244777,-0.649240,1.132522,0.393771,1.275408,0.626316,-0.155323,1.321870
4,201944,3,30,8,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,...,1.369037,-0.786588,-0.708788,-0.300341,-0.918274,-0.802821,-0.923853,-1.057658,-1.173488,1.397554,-0.690952,-0.743108,-0.347666,-0.993071,-0.732591,0.591077,-0.435466,-1.191682,0.864740,-0.127781,0.834426,0.657916,0.277175,-0.844201,-0.443121,1.231116,-0.133753,1.275345,0.194132,0.825412,0.111862,1.101646,1.331354,-0.568684,1.260121,0.518600,1.228647,-0.419258,-1.330446,1.066770


In [0]:
# Extract GEO-Embeds for all test data points.
# Then assign with test_data

out = []
for dat in geo3[260601:]:
    layer_output = get_int_layer_output([[dat]])[0]
    out.append(layer_output)

out = np.array(out)
out = np.squeeze(out)

test_data = pd.get_dummies(test_x.copy())
test_data = test_data.drop(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'], axis=1)
test_data = test_data.assign(geo_feat1=out[:,0],
                               geo_feat2=out[:,1],
                               geo_feat3=out[:,2],  
                               geo_feat4=out[:,3],
                               geo_feat5=out[:,4],    
                               geo_feat6=out[:,5],
                               geo_feat7=out[:,6],
                               geo_feat8=out[:,7],
                               geo_feat9=out[:,8],
                               geo_feat10=out[:,9],
                               geo_feat11=out[:,10],
                               geo_feat12=out[:,11],
                               geo_feat13=out[:,12],
                               geo_feat14=out[:,13],
                               geo_feat15=out[:,14],           
                               geo_feat16=out[:,15],
                               geo_feat17=out[:,16],
                               geo_feat18=out[:,17],
                               geo_feat19=out[:,18],
                               geo_feat20=out[:,19],
                               geo_feat21=out[:,20],
                               geo_feat22=out[:,21])'''
                               geo_feat23=out[:,22],
                               geo_feat24=out[:,23],
                               geo_feat25=out[:,24],
                               geo_feat26=out[:,25],
                               geo_feat27=out[:,26],
                               geo_feat28=out[:,27],
                               geo_feat29=out[:,28],
                               geo_feat30=out[:,29],
                               geo_feat31=out[:,30],
                               geo_feat32=out[:,31],
                               geo_feat33=out[:,32],
                               geo_feat34=out[:,33],
                               geo_feat35=out[:,34],
                               geo_feat36=out[:,35],
                               geo_feat37=out[:,36],
                               geo_feat38=out[:,37],
                               geo_feat39=out[:,38],
                               geo_feat40=out[:,39],
                               geo_feat41=out[:,40],
                               geo_feat42=out[:,41],
                               geo_feat43=out[:,42],
                               geo_feat44=out[:,43],
                               geo_feat45=out[:,44],
                               geo_feat46=out[:,45],
                               geo_feat47=out[:,46],
                               geo_feat48=out[:,47],
                               geo_feat49=out[:,48],
                               geo_feat50=out[:,49],
                               geo_feat51=out[:,50],
                               geo_feat52=out[:,51])    '''

In [66]:
from google.colab import drive
drive.mount('drive')
test_data.to_csv('test_data.csv')
!cp test_data.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [64]:
from google.colab import drive
drive.mount('drive')
train_data.to_csv('train_data.csv')
!cp train_data.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
def threshold_arr(array):
    # Get major confidence-scored predicted value.
    new_arr = []
    for ix, val in enumerate(array):
        loc = np.array(val).argmax(axis=0)
        k = list(np.zeros((len(val))))
        k[loc]=1
        new_arr.append(k)
        
    return np.array(new_arr)

In [51]:
y = np.array(train_y["damage_grade"])-1

df = train_data.drop(["building_id"], axis=1)
x = np.array(df)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for ix, (train_index, test_index) in enumerate(kf.split(x)):
    lgb_params = {
        "objective" : "multiclass",
        "num_class":3,
        "metric" : "multi_error",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "feature_fraction" : 0.5,
        "min_sum_hessian_in_leaf" : 0.1,
        "max_bin":8192,
        "verbosity" : 1,
        "num_threads":6,
      
    }

    X_train, X_test, y_train, y_test= x[train_index], x[test_index], y[train_index], y[test_index]

    new_train_data = lgb.Dataset(X_train, label=y_train)
    new_test_data  = lgb.Dataset(X_test, label=y_test)

    lgb_clf = lgb.train(lgb_params,
                        new_train_data,
                        20000,
                        valid_sets = [new_test_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 100)

    y_test_pred = lgb_clf.predict(X_test)
    print("F1-MICRO SCORE: ", f1_score(np.array(pd.get_dummies(y_test)), threshold_arr(y_test_pred), average='micro'))
    

Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's multi_error: 0.276165
[200]	valid_0's multi_error: 0.265287
[300]	valid_0's multi_error: 0.259953
[400]	valid_0's multi_error: 0.256595
[500]	valid_0's multi_error: 0.255118
[600]	valid_0's multi_error: 0.253832
[700]	valid_0's multi_error: 0.252374
[800]	valid_0's multi_error: 0.252067
[900]	valid_0's multi_error: 0.251453
[1000]	valid_0's multi_error: 0.25107
[1100]	valid_0's multi_error: 0.250974
[1200]	valid_0's multi_error: 0.250552
[1300]	valid_0's multi_error: 0.249688
[1400]	valid_0's multi_error: 0.250206
[1500]	valid_0's multi_error: 0.249842
[1600]	valid_0's multi_error: 0.249842
[1700]	valid_0's multi_error: 0.249899
[1800]	valid_0's multi_error: 0.249611
[1900]	valid_0's multi_error: 0.249957
[2000]	valid_0's multi_error: 0.249688
[2100]	valid_0's multi_error: 0.249611
[2200]	valid_0's multi_error: 0.248921
[2300]	valid_0's multi_error: 0.249113
[2400]	valid_0's multi_error: 0.249113
[2500]	val